In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
#from googletrans import Translator
from deep_translator import GoogleTranslator

In [2]:
# Load cherry blossom bloom dates from JMA
url = 'https://www.data.jma.go.jp/sakura/data/sakura003_04.html'
response = requests.get(url)
data = response.content

In [3]:
# Parse HTML with BeautifulSoup
soup = BeautifulSoup(data, 'lxml')

In [41]:
# Print the entire HTML to inspect the structure
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ja" xml:lang="ja" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   気象庁 | さくらの開花日(1991-2000年)
  </title>
  <meta content="気象庁 Japan Meteorological Agency" name="Author"/>
  <meta content="気象庁 Japan Meteorological Agency" name="keywords"/>
  <meta content="過去のさくらの開花日・満開日を表示します" name="description"/>
  <meta content="text/css" http-equiv="Content-Style-Type"/>
  <meta content="text/javascript" http-equiv="Content-Script-Type"/>
  <link href="/com/css/define.css" media="all" rel="stylesheet" type="text/css"/>
  <link href="css/default.css" media="all" rel="stylesheet" type="text/css"/>
  <script src="//www.jma.go.jp/jma/com/js/jquery.js" type="text/javascript">
  </script>
  <style media="all" type="text/css">
   <!-- @import url(/com/default.css); -->
  </style>
 </head>
 <body>

In [42]:
# Find the table and extract rows
# Find the pre tag containing the table data
pre_tag = soup.find('pre')

In [4]:
# Find the pre tag containing the table data
pre_tag = soup.find('pre')
# Extract the text from the pre tag
table_text = pre_tag.get_text()

# Print the extracted table text
#print(table_text)

In [5]:
# Process the table text to extract data
lines = table_text.split('\n')
data = []
columns = ['Location', '*']
for year in range(1991,2001):
    columns.extend([f'{year}_Month', f'{year}_Day'])
    
columns.extend(['平年値_Month', '平年値_Day', '代替種目'])

In [6]:
# Extract data rows
for line in lines:
    if line.strip() and not line.startswith('地点名') and not line.startswith('月'):
        row_data = line.split()
        #if len(row_data) == 25:  # Ensure the row has the correct number of columns
        data.append(row_data)

In [7]:
# For some reason the previous code still didn't eliminate the lines starting with month/day so we filter out the data list instead
# Initialize an empty list to store valid data rows
data_filtered = []

# Extract data rows
for row_data in data:
    # Check if the row does not start with '月' or '地点名'
    if not row_data[0].startswith('月') and not row_data[0].startswith('地点名'):
        data_filtered.append(row_data)

# Now data_filtered contains only valid rows excluding rows starting with '月' or '地点名'
#print(data_filtered)

In [8]:
# Function to insert 'Not currently under observation' as the second element where '*' is missing
def insert_star(lst):
    if lst[1] != '*':
        lst.insert(1, 'Not currently under observation')
    return lst

# Apply the function to each list in data
modified_data = [insert_star(row) for row in data_filtered]

In [9]:
# Now that we fixed that I also noticed that the observations where the date is missing are filled out with a '-' value
# however because these values are present only under the 'day' columns and this will create a problem later on when we
# try to combine the day and month values, lets try and fix that

modified_data2 = []

# Iterate through each row in data_filtered
for row_data in modified_data:
    modified_row = []
    for item in row_data:
        # Check if the item is '-'
        if item == '-':
            modified_row.append('-')  # Add '-' before existing '-'
            modified_row.append(item) 
        else:
            modified_row.append(item)  # Add the item unchanged
        
    # Append the modified row to data_modified
    modified_data2.append(modified_row)

# Now data_modified contains all rows with '-' prefixed where necessary
#print(modified_data2)


In [10]:
# Create a DataFrame from the extracted data
df = pd.DataFrame(modified_data2, columns=columns)

In [11]:
df.head()

,Location,*,1991_Month,1991_Day,1992_Month,1992_Day,1993_Month,1993_Day,1994_Month,1994_Day,...,1997_Day,1998_Month,1998_Day,1999_Month,1999_Day,2000_Month,2000_Day,平年値_Month,平年値_Day,代替種目
0,稚内,*,5,14,5,17,5,15,5,16,...,15,5,11,5,14,5,18,5,13,えぞやまざくら
1,留萌,Not currently under observation,5,7,5,12,5,12,5,14,...,12,4,29,5,14,5,13,-,-,えぞやまざくら
2,旭川,*,5,5,5,9,5,8,5,11,...,5,4,23,5,4,5,9,5,4,えぞやまざくら
3,網走,*,5,8,5,13,5,20,5,13,...,12,4,28,5,9,5,18,5,10,えぞやまざくら
4,札幌,*,5,2,5,4,5,6,5,6,...,1,4,23,5,3,5,8,5,1,None


In [12]:
# we will drop the 'average value' and 'alternative species' bcz we will not be needing them
df = df.iloc[:, :-3]

In [13]:
# Combine month and day columns for each year
years = list(range(1991, 2001))  # Years from 1991 to 2000
for year in years:
    month_col = f'{year}_Month'
    day_col = f'{year}_Day'
    combined_col = f'{year}'
    df[combined_col] = df[month_col] + ' ' + df[day_col]
    df.drop([month_col, day_col], axis=1, inplace=True)

In [14]:
# attetion ! now we have '- -' values where the date is missing lets replace them with NaN
df.replace('- -', np.nan, inplace=True)

In [15]:
df.head()

,Location,*,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,稚内,*,5 14,5 17,5 15,5 16,5 8,5 20,5 15,5 11,5 14,5 18
1,留萌,Not currently under observation,5 7,5 12,5 12,5 14,5 5,5 20,5 12,4 29,5 14,5 13
2,旭川,*,5 5,5 9,5 8,5 11,5 2,5 7,5 5,4 23,5 4,5 9
3,網走,*,5 8,5 13,5 20,5 13,5 8,5 24,5 12,4 28,5 9,5 18
4,札幌,*,5 2,5 4,5 6,5 6,5 1,5 5,5 1,4 23,5 3,5 8


In [16]:
df.tail()

,Location,*,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
97,久米島,Not currently under observation,1 2,1 5,1 7,1 12,1 15,1 13,1 15,1 20,1 26,1 7
98,那覇,*,1 24,1 26,1 23,1 20,2 8,1 25,1 20,1 16,2 2,1 18
99,名護,Not currently under observation,1 7,1 6,1 4,1 10,1 16,1 4,1 1,1 4,12 28#,1 1
100,西表島,Not currently under observation,1 27,1 21,1 14,1 21,2 3,1 14,1 20,1 30,2 7,1 16
101,南大東島,*,1 22,1 20,1 4,1 7,1 19,1 25,1 24,1 13,1 27,1 17


In [17]:
# before we continue, it looks like one observation is wrong lets fix that
df['1999'] = df['1999'].replace({'12 28#': '12 28'})

In [18]:
# Loop through each year column (adjust range according to your columns)
for year in range(1991, 2001):  # assuming columns are from '1991_Month' to '2000_Day'
    # Replace any leading spaces (if any)
    df[f'{year}'] = df[f'{year}'].str.replace(r'^\s+', '', regex=True)
    
    # Fixing the format to mm-dd
    df[f'{year}'] = pd.to_datetime(df[f'{year}'], format='%m %d').dt.strftime('%m-%d')

In [19]:
df.head()

,Location,*,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,稚内,*,05-14,05-17,05-15,05-16,05-08,05-20,05-15,05-11,05-14,05-18
1,留萌,Not currently under observation,05-07,05-12,05-12,05-14,05-05,05-20,05-12,04-29,05-14,05-13
2,旭川,*,05-05,05-09,05-08,05-11,05-02,05-07,05-05,04-23,05-04,05-09
3,網走,*,05-08,05-13,05-20,05-13,05-08,05-24,05-12,04-28,05-09,05-18
4,札幌,*,05-02,05-04,05-06,05-06,05-01,05-05,05-01,04-23,05-03,05-08


In [22]:
def translate_text(text):
    try:
        translated = GoogleTranslator(source='ja', target='en').translate(text)
        return translated
    except Exception as e:
        print(f"Translation error: {e}")
        return text  # Return original text on error

# Apply translation to the DataFrame column
df['Location'] = df['Location'].apply(translate_text)

In [23]:
df.head()

,Location,*,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,Wakkanai,*,05-14,05-17,05-15,05-16,05-08,05-20,05-15,05-11,05-14,05-18
1,Rumoi,Not currently under observation,05-07,05-12,05-12,05-14,05-05,05-20,05-12,04-29,05-14,05-13
2,Asahikawa,*,05-05,05-09,05-08,05-11,05-02,05-07,05-05,04-23,05-04,05-09
3,Abashiri,*,05-08,05-13,05-20,05-13,05-08,05-24,05-12,04-28,05-09,05-18
4,Sapporo,*,05-02,05-04,05-06,05-06,05-01,05-05,05-01,04-23,05-03,05-08


In [ ]:
# check if this data is first bloom or full bloom

In [30]:
# check for any duplicates
a = list(df['Location'].unique())
print(a.sort())
print(a)
print(len(a))
print(df.shape)

None
['Abashiri', 'Aikawa', 'Akita', 'Aomori', 'Asahikawa', 'Cape Shionomisaki', 'Choshi', 'Esashi', 'Fukue', 'Fukui', 'Fukuoka', 'Fukushima', 'Gifu', 'Hachijojima', 'Hachinohe', 'Hakodate', 'Hamada', 'Hamamatsu', 'Hikone', 'Hiroo', 'Hiroshima', 'Iida', 'Iriomote Island', 'Ishigaki Island', 'Iwamizawa', 'Izuhara', 'Kagoshima', 'Kanazawa', 'Kobe', 'Kochi', 'Kofu', 'Kumagaya', 'Kumamoto', 'Kumejima', 'Kushiro', 'Kutchan', 'Kyoto', 'Maebashi', 'Maizuru', 'Matsue', 'Matsumoto', 'Matsuyama', 'Minamidaito Island', 'Mito', 'Miyake Island', 'Miyako', 'Miyakojima', 'Miyazaki', 'Monbetsu', 'Morioka', 'Muroran', 'Nagano', 'Nagasaki', 'Nago', 'Nagoya', 'Naha', 'Nara', 'Naze', 'Nemuro', 'Niigata', 'Nobeoka', 'Obihiro', 'Oita', 'Okayama', 'Onahama', 'Osaka', 'Oshima', 'Owase', 'Rumoi', 'Saga', 'Saigo', 'Sakata', 'Sapporo', 'Sendai', 'Shimonoseki', 'Shinjo', 'Shirakawa', 'Shizuoka', 'Sumoto', 'Takada', 'Takamatsu', 'Takayama', 'Tanegashima', 'Tateyama', 'Tokushima', 'Tokyo', 'Tottori', 'Toyama', 'Toy

In [ ]:
# check the GMU git hub , only need to add lat long of locations, and maybe the data format they used is better in japan.csv
# also maybe add temperature data